In [19]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from glob import glob # pour trouver les fichiers de tracking
import multiprocessing # pour le traitement parallèle
from functools import partial # pour faciliter l'utilisation de la fonction avec des arguments partiels
#import ffmpeg

from pyproj import Transformer


In [20]:
pip install pyproj

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Les données LPS

In [61]:
fusion_3 = "../../DATABASE/basket/LPS_sample/SAMPLE_3_fusion.csv"
fusion_4 = "../../DATABASE/basket/LPS_sample/SAMPLE_4_fusion.csv"
fusion_5 = "../../DATABASE/basket/LPS_sample/SAMPLE_5_fusion.csv"
fusion_7 = "../../DATABASE/basket/LPS_sample/SAMPLE_7_fusion.csv"
fusion_10 = "../../DATABASE/basket/LPS_sample/SAMPLE_10_fusion.csv"
fusion_15 = "../../DATABASE/basket/LPS_sample/SAMPLE_15_fusion.csv"
fusion_16 = "../../DATABASE/basket/LPS_sample/SAMPLE_16_fusion.csv"
fusion_21 = "../../DATABASE/basket/LPS_sample/SAMPLE_21_fusion.csv"
fusion_22 = "../../DATABASE/basket/LPS_sample/SAMPLE_22_fusion.csv"
fusion_23 = "../../DATABASE/basket/LPS_sample/SAMPLE_23_fusion.csv"
fusion_29 = "../../DATABASE/basket/LPS_sample/SAMPLE_29_fusion.csv"


In [62]:
lps_3 = pd.read_csv(fusion_3, sep=";")
lps_4 = pd.read_csv(fusion_4, sep=";")
lps_5 = pd.read_csv(fusion_5, sep=";")
lps_7 = pd.read_csv(fusion_7, sep=";")
lps_10 = pd.read_csv(fusion_10, sep=";")
lps_15 = pd.read_csv(fusion_15, sep=";")
lps_16 = pd.read_csv(fusion_16, sep=";")
lps_21 = pd.read_csv(fusion_21, sep=";")
lps_22 = pd.read_csv(fusion_22, sep=";")
lps_23 = pd.read_csv(fusion_23, sep=";")
lps_29 = pd.read_csv(fusion_29, sep=";")

In [63]:
lps_3.head(5)

,timestamp,lat_brute,long_brute,latitude_fusion,longitude_fusion,hdop,vitesse_fusion,battery
0,1702478113100,NaN,NaN,NaN,NaN,-1.0,-1.0,0
1,1702478113300,NaN,NaN,NaN,NaN,-1.0,-1.0,0
2,1702478113500,NaN,NaN,NaN,NaN,-1.0,-1.0,0
3,1702478113700,NaN,NaN,NaN,NaN,-1.0,-1.0,0
4,1702478113900,NaN,NaN,NaN,NaN,-1.0,-1.0,0


In [64]:
def nettoyage_lps(df):
    columns_to_drop = ["lat_brute", "long_brute", "hdop", "vitesse_fusion", "battery"]
    df = df.drop(columns=columns_to_drop, errors='ignore')
    transformer = Transformer.from_crs("epsg:4326", "epsg:2154", always_xy=True)
    df['x'], df['y'] = transformer.transform(df['longitude_fusion'].values, df['latitude_fusion'].values)
    df['x_norm'] = df['x'] - df['x'].mean()
    df['y_norm'] = df['y'] - df['y'].mean()
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    start_time = df['datetime'].iloc[0]
    df['relative_time'] = (df['datetime'] - start_time).dt.total_seconds()
    df = df.dropna(subset=["latitude_fusion", "longitude_fusion"])
    df = df.reset_index(drop=True)
    return df

In [65]:
lps_3 = nettoyage_lps(lps_3)
#lps_3[0:100:2]

Les fichiers lps ont été enregistrés de 14h35 a 17h01. Donc deux heures 25 min de jeu.

### Les données tracking

In [103]:
tracking_1_1_2_1 = "../../DATABASE/basket/TRACKING/Equipe1_Vague1_Poss2_video_1.txt"
tracking_df = pd.read_csv(tracking_1_1_2_1, names=['frame', 'player_id', 'x', 'y'], sep=",")

In [104]:
tracking_df

,frame,player_id,x,y
0,0,0,10.738531,2.423311
1,0,1,10.758081,7.556952
2,0,2,9.685260,7.294818
3,0,3,6.877824,9.319980
4,0,4,5.604179,9.292738
...,...,...,...,...
3282,385,3,5.606390,1.841755
3283,385,4,0.658973,5.352539
3284,385,6,2.336708,9.233820
3285,385,7,2.905753,8.886047


In [105]:
def nettoyage_tracking(df):
    df = df.dropna(subset=["x", "y"])
    df = df.reset_index(drop=True)
    df["time"] = df["frame"] / 25
    df["x"] = df["x"].astype(float)
    df["y"] = df["y"].astype(float)
    df["player_id"] = df["player_id"].astype(int)
    df["frame"] = df["frame"].astype(int)

    df["x_norm"] = df.groupby("player_id")["x"].transform(lambda x: (x - x.mean()) / x.std() if x.std() > 0 else 0)
    df["y_norm"] = df.groupby("player_id")["y"].transform(lambda y: (y - y.mean()) / y.std() if y.std() > 0 else 0)
    return df

In [106]:
tracking_df =nettoyage_tracking(tracking_df)
tracking_df

,frame,player_id,x,y,time,x_norm,y_norm
0,0,0,10.738531,2.423311,0.0,-2.531977,-1.515850
1,0,1,10.758081,7.556952,0.0,0.892907,-1.034344
2,0,2,9.685260,7.294818,0.0,1.033218,-1.134836
3,0,3,6.877824,9.319980,0.0,0.266733,0.060258
4,0,4,5.604179,9.292738,0.0,1.448783,0.653209
...,...,...,...,...,...,...,...
3282,385,3,5.606390,1.841755,15.4,-0.779155,-1.945877
3283,385,4,0.658973,5.352539,15.4,-2.794906,-3.013042
3284,385,6,2.336708,9.233820,15.4,-2.416915,2.845964
3285,385,7,2.905753,8.886047,15.4,-2.279900,3.456345


In [ ]:
tracking_df_echte = tracking_df[]

In [107]:
joueur_3 = tracking_df[tracking_df["player_id"] == 3]
joueur_3 = joueur_3.reset_index(drop=True)
joueur_3

,frame,player_id,x,y,time,x_norm,y_norm
0,0,3,6.877824,9.319980,0.00,0.266733,0.060258
1,1,3,6.969496,9.305518,0.04,0.342142,0.056378
2,2,3,6.992512,9.343356,0.08,0.361075,0.066529
3,3,3,7.031557,9.400229,0.12,0.393194,0.081786
4,4,3,7.040776,9.476280,0.16,0.400778,0.102187
...,...,...,...,...,...,...,...
381,381,3,5.730844,1.894935,15.24,-0.676779,-1.931611
382,382,3,5.698042,1.883440,15.28,-0.703762,-1.934694
383,383,3,5.661434,1.857994,15.32,-0.733876,-1.941521
384,384,3,5.639479,1.852672,15.36,-0.751936,-1.942949


In [108]:
joueur_3[:20]

,frame,player_id,x,y,time,x_norm,y_norm
0,0,3,6.877824,9.319980,0.00,0.266733,0.060258
1,1,3,6.969496,9.305518,0.04,0.342142,0.056378
2,2,3,6.992512,9.343356,0.08,0.361075,0.066529
3,3,3,7.031557,9.400229,0.12,0.393194,0.081786
4,4,3,7.040776,9.476280,0.16,0.400778,0.102187
5,5,3,7.041498,9.532124,0.20,0.401372,0.117168
6,6,3,7.054055,9.629006,0.24,0.411701,0.143158
7,7,3,7.047019,9.729725,0.28,0.405913,0.170177
8,8,3,7.041348,9.830388,0.32,0.401249,0.197182
9,9,3,7.034868,9.972550,0.36,0.395918,0.235319


### Let's test something

In [73]:
def experience1_sans_normalisation(fusion_df, tracking_df, step=1):
    
    best_result = {"best_player": None, "best_start_time": None, "best_distance": float("inf")}

    for pid in tracking_df["player_id"].unique():
        joueur_df = tracking_df[tracking_df["player_id"] == pid].sort_values("time")
        if len(joueur_df) < 10:
            continue

        t_min, t_max = joueur_df["time"].min(), joueur_df["time"].max()
        max_start = fusion_df["relative_time"].max() - (t_max - t_min)

        for start_time in np.arange(0, max_start, step):
            end_time = start_time + (t_max - t_min)
            lps_window = fusion_df[(fusion_df["relative_time"] >= start_time) &
                                   (fusion_df["relative_time"] <= end_time)]

            if len(lps_window) < 10:
                continue

            try:
                interp_x = np.interp(lps_window["relative_time"] - start_time, joueur_df["time"] - t_min, joueur_df["x"])
                interp_y = np.interp(lps_window["relative_time"] - start_time, joueur_df["time"] - t_min, joueur_df["y"])
            except:
                continue

            dist = np.sqrt((interp_x - lps_window["latitude_fusion"].values)**2 +
                           (interp_y - lps_window["longitude_fusion"].values)**2)
            mean_dist = dist.mean()

            if mean_dist < best_result["best_distance"]:
                best_result = {
                    "best_player": pid,
                    "best_start_time": start_time,
                    "best_distance": mean_dist
                }

    return best_result

In [74]:
def tester_tous_les_trackings(fusion_df, TRACKING):
    fusion_df = nettoyage_lps(fusion_df)
    results = []

    for file in os.listdir(TRACKING):
        if not file.endswith(".txt"):
            continue

        tracking_path = os.path.join(TRACKING, file)
        try:
            tracking_df = pd.read_csv(tracking_path, names=["frame", "player_id", "x", "y"], sep=",")
            tracking_df = nettoyage_tracking(tracking_df)
            result = experience1_sans_normalisation(fusion_df, tracking_df)
        except:
            result = {"best_player": None, "best_start_time": None, "best_distance": None}

        results.append({
            "fichier_tracking": file,
            "joueur_id": result["best_player"],
            "start_time_LPS": result["best_start_time"],
            "distance_moyenne": result["best_distance"]
        })

    return pd.DataFrame(results)

#### TESTER TOUS LES FICHIERS

In [75]:
TRACKING = "../../DATABASE/basket/TRACKING/"
#resultats = tester_tous_les_trackings(fusion_df= lps_3, TRACKING=TRACKING)
#resultats

In [76]:
#resultats.loc[resultats["distance_moyenne"].idxmin()]

TEST PAR FICHIER

In [77]:
lps_3 = nettoyage_lps(lps_3)
resultats = experience1_sans_normalisation(fusion_df=lps_3, tracking_df=tracking_df)
player_id = resultats["best_player"]
start_time = resultats['best_start_time']

### NOUVEAU TEST AVEC NORMALISATION

In [99]:
def experience2_avec_normalisation(fusion_df, tracking_df, step=1):
    # cette transformat° est necessaire pour passer du longi et lat en coord metriques (x,y)
    transformer = Transformer.from_crs("epsg:4326", "epsg:2154", always_xy=True)
    fusion_df = fusion_df.copy()
    fusion_df['x'], fusion_df['y'] = transformer.transform(
        fusion_df['longitude_fusion'].values, fusion_df['latitude_fusion'].values
    )

    print("Fusion DataFrame apres transformation:")
    print(fusion_df.head())

    best_result = {"best_player": None, "best_start_time": None, "best_distance": float("inf")}

    for pid in tracking_df["player_id"].unique():
        joueur_df = tracking_df[tracking_df["player_id"] == pid].sort_values("time")
        if len(joueur_df) < 10:
            continue

        t_min, t_max = joueur_df["time"].min(), joueur_df["time"].max()
        traj_duration = t_max - t_min
        max_start = fusion_df["relative_time"].max() - traj_duration

        for start_time in np.arange(0, max_start, step):
            end_time = start_time + traj_duration
            lps_window = fusion_df[(fusion_df["relative_time"] >= start_time) &
                                   (fusion_df["relative_time"] <= end_time)]

            if len(lps_window) < 10:
                continue
            # Les données LPS sont collectées à 100 Hz et celles du tracking à 25 Hz, donc 
            #Les instants ne correspondent pas exactement. Pour comparer deux trajectoires, il faut les évaluer aux mêmes temps ➜ d’où l’interpolation.
            try:
                interp_x = np.interp(lps_window["relative_time"] - start_time,
                                     joueur_df["time"] - t_min, joueur_df["x"])
                interp_y = np.interp(lps_window["relative_time"] - start_time,
                                     joueur_df["time"] - t_min, joueur_df["y"])
                #Une trajectoire (x, y) du joueur estimée sur les mêmes temps que les LPS
            except Exception:
                continue

            # Normalisation des deux trajectoires pour comparer les formes des trajectoires indépendamment de l’échelle et de 
            #la position (par exemple, s’ils bougent de la même manière mais pas au même endroit), on centre et réduit les deux courbes (tracking et LPS)
            gps_mean_x, gps_std_x = interp_x.mean(), interp_x.std()
            gps_mean_y, gps_std_y = interp_y.mean(), interp_y.std()
            lps_x = lps_window["x"].values
            lps_y = lps_window["y"].values
            lps_mean_x, lps_std_x = lps_x.mean(), lps_x.std()
            lps_mean_y, lps_std_y = lps_y.mean(), lps_y.std()

            if min(gps_std_x, gps_std_y, lps_std_x, lps_std_y) == 0:
                continue

            norm_interp_x = (interp_x - gps_mean_x) / gps_std_x
            norm_interp_y = (interp_y - gps_mean_y) / gps_std_y
            norm_lps_x = (lps_x - lps_mean_x) / lps_std_x
            norm_lps_y = (lps_y - lps_mean_y) / lps_std_y

            rmse = np.sqrt(np.mean((norm_interp_x - norm_lps_x)**2 + (norm_interp_y - norm_lps_y)**2))

            if rmse < best_result["best_distance"]:
                best_result = {
                    "best_player": pid,
                    "best_start_time": start_time,
                    "best_distance": rmse
                }

    return best_result


In [79]:
#resultats = experience2_avec_normalisation(fusion_df=lps_3, tracking_df=tracking_df)

In [80]:
resultats

{'best_player': None, 'best_start_time': None, 'best_distance': inf}

### CREATION DE L'ANIMATION POUR LPS

In [81]:
def create_lps_animation(lps_df, start_time, duration=20, save_path="lps_animation.gif"):
    """
    Crée une animation GIF des positions LPS à partir d'un temps de début donné.

    Args:
        lps_df: DataFrame LPS avec colonnes ['relative_time', 'x', 'y']
        start_time: temps de début (en secondes, relatif)
        duration: durée de l'animation en secondes (par défaut 20)
        save_path: chemin du fichier GIF à sauvegarder
    """
    # Filtrer la fenêtre temporelle
    end_time = start_time + duration
    window = lps_df[(lps_df['relative_time'] >= start_time) & (lps_df['relative_time'] <= end_time)].copy()
    window = window.dropna(subset=['x', 'y'])
    if window.empty:
        print("Aucune donnée LPS dans la fenêtre temporelle spécifiée.")
        return

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_facecolor('black')
    ax.grid(True, alpha=0.3, color='white')
    ax.set_xlabel('Latitude', fontsize=12, color='white')
    ax.set_ylabel('Longitude', fontsize=12, color='white')
    ax.tick_params(colors='white')

    x_min, x_max = window['x'].min() - 0.0001, window['x'].max() + 0.0001
    y_min, y_max = window['y'].min() - 0.0001, window['y'].max() + 0.0001
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    title = ax.text(0.5, 1.05, 'Animation des Positions LPS',
                    transform=ax.transAxes, ha='center', va='bottom',
                    fontsize=16, fontweight='bold', color='white')

    time_text = ax.text(0.02, 0.98, '', transform=ax.transAxes,
                        fontsize=12, color='cyan', verticalalignment='top',
                        bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))

    dot, = ax.plot([], [], 'o', markersize=10, color='orange', markeredgecolor='white', markeredgewidth=2, zorder=10)
    trail, = ax.plot([], [], '-', alpha=0.7, linewidth=2, color='orange')

    times = window['relative_time'].values
    total_frames = len(times)
    interval = (duration * 1000) / total_frames if total_frames > 0 else 50

    def animate(frame):
        t = times[frame]
        current = window.iloc[frame]
        dot.set_data([current['x']], [current['y']])
        trail.set_data(window['x'].values[:frame+1], window['y'].values[:frame+1])
        time_text.set_text(f"t = {t:.2f} s")
        return dot, trail, time_text

    anim = FuncAnimation(fig, animate, frames=total_frames, interval=interval, blit=True, repeat=True)
    anim.save(save_path, writer='pillow', fps=total_frames/duration)
    plt.close(fig)
    print(f"Animation LPS sauvegardée sous {save_path}")

#lps_3 = nettoyage_lps(lps_3)
#create_lps_animation(lps_3, start_time=6601.0, duration=20, save_path="../../DATABASE/animation/lps_3_joueur_track_3_test.gif")

### CREATION DE L'ANIMATION POUR LE TRACKING

In [82]:
def create_tracking_animation(df, duration=20, save_path="animation.gif"):
    """
    Crée une animation dynamique des données de tracking sans afficher les trajectoires et sauvegarde en GIF.

    Args:
        df: DataFrame avec les colonnes ['time', 'player_id', 'x', 'y']
        duration: Durée de l'animation en secondes
        save_path: Chemin du fichier GIF à sauvegarder
    """

    fig, ax = plt.subplots(figsize=(12, 8))
    fig.patch.set_facecolor('black')

    x_min, x_max = df['x'].min() - 1, df['x'].max() + 1
    y_min, y_max = df['y'].min() - 1, df['y'].max() + 1

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_facecolor('darkslategray')
    ax.grid(True, alpha=0.3, color='white')
    ax.set_xlabel('Position X', fontsize=12, color='white')
    ax.set_ylabel('Position Y', fontsize=12, color='white')
    ax.tick_params(colors='white')

    title = ax.text(0.5, 1.05, 'Animation des Positions des Joueurs',
                    transform=ax.transAxes, ha='center', va='bottom',
                    fontsize=16, fontweight='bold', color='white')

    time_text = ax.text(0.02, 0.98, '', transform=ax.transAxes,
                        fontsize=12, color='cyan', verticalalignment='top',
                        bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))

    unique_players = sorted(df['player_id'].unique())
    colors = plt.cm.tab10(np.linspace(0, 1, len(unique_players)))
    player_colors = dict(zip(unique_players, colors))

    player_dots = {}
    player_labels = {}

    for player_id in unique_players:
        dot, = ax.plot([], [], 'o', markersize=12,
                       color=player_colors[player_id],
                       markeredgecolor='white', markeredgewidth=2,
                       zorder=10)
        player_dots[player_id] = dot

        label = ax.text(0, 0, f'{player_id}', fontsize=8, ha='center', va='center',
                        color='white', fontweight='bold', zorder=11)
        player_labels[player_id] = label

    legend_elements = [plt.Line2D([0], [0], marker='o', color='w',
                                  markerfacecolor=player_colors[pid], markersize=8,
                                  label=f'Joueur {pid}', markeredgecolor='white')
                       for pid in unique_players]
    ax.legend(handles=legend_elements, loc='upper right', facecolor='black',
              edgecolor='white', labelcolor='white')

    times = sorted(df['time'].unique())
    total_frames = len(times)
    interval = (duration * 1000) / total_frames

    def animate(frame):
        current_time = times[frame]
        current_data = df[df['time'] == current_time]

        time_text.set_text(f'time: {current_time:.2f}')

        for player_id in unique_players:
            player_data = current_data[current_data['player_id'] == player_id]

            if not player_data.empty:
                x, y = player_data.iloc[0]['x'], player_data.iloc[0]['y']
                player_dots[player_id].set_data([x], [y])
                player_labels[player_id].set_position((x, y))
            else:
                player_dots[player_id].set_data([], [])
                player_labels[player_id].set_position((x_min-10, y_min-10))

        return list(player_dots.values()) + list(player_labels.values()) + [time_text]

    anim = FuncAnimation(fig, animate, frames=total_frames,
                         interval=interval, blit=True, repeat=True)

    # Sauvegarde en GIF
    anim.save(save_path, writer='pillow', fps=total_frames/duration)
    plt.close(fig)
    print(f"Animation sauvegardée sous {save_path}")

# Exemple d'utilisation :
#create_tracking_animation(tracking_df, duration=20, save_path="../../DATABASE/animation/tracking_animation_3.gif")


### TEST DE LSAP

In [100]:
from scipy.optimize import linear_sum_assignment

def experience2_avec_LSAP(fusion_df, tracking_df, step=1):
    best_result = {"best_player": None, "best_start_time": None, "best_distance": float("inf")}

    for pid in tracking_df["player_id"].unique():
        joueur_df = tracking_df[tracking_df["player_id"] == pid].sort_values("time")
        if len(joueur_df) < 10:
            continue

        t_min, t_max = joueur_df["time"].min(), joueur_df["time"].max()
        traj_duration = t_max - t_min
        max_start = fusion_df["relative_time"].max() - traj_duration

        for start_time in np.arange(0, max_start, step):
            end_time = start_time + traj_duration
            lps_window = fusion_df[(fusion_df["relative_time"] >= start_time) &
                                   (fusion_df["relative_time"] <= end_time)]

            if len(lps_window) < 10:
                continue

            try:
                interp_x = np.interp(
                    lps_window["relative_time"] - start_time,
                    joueur_df["time"] - t_min,
                    joueur_df["x"]
                )
                interp_y = np.interp(
                    lps_window["relative_time"] - start_time,
                    joueur_df["time"] - t_min,
                    joueur_df["y"]
                )
            except Exception:
                continue

            tracking_interp = np.stack((interp_x, interp_y), axis=1)
            lps_coords = lps_window[['x', 'y']].to_numpy()

            if tracking_interp.shape[0] != lps_coords.shape[0]:
                continue

            def normalize_traj(coords):
                mean = coords.mean(axis=0)
                std = coords.std(axis=0)
                if np.any(std == 0):
                    return None
                return (coords - mean) / std

            norm_tracking = normalize_traj(tracking_interp)
            norm_lps = normalize_traj(lps_coords)
            if norm_tracking is None or norm_lps is None:
                continue

            #Matrice de distances entre les points normalisés ===
            cost_matrix = np.linalg.norm(
                norm_lps[:, np.newaxis, :] - norm_tracking[np.newaxis, :, :],
                axis=2
            )

            #résolution du LSAP
            row_ind, col_ind = linear_sum_assignment(cost_matrix)

            #Distance finale (RMSE sur points associés)
            total_dist = np.sum((norm_lps[row_ind] - norm_tracking[col_ind]) ** 2)
            rmse = np.sqrt(total_dist / len(row_ind))

            if rmse < best_result["best_distance"]:
                best_result = {
                    "best_player": pid,
                    "best_start_time": start_time,
                    "best_distance": rmse
                }

    return best_result, cost_matrix, row_ind, col_ind


In [101]:
result, cost_matrix, row_ind, col_ind = experience2_avec_LSAP(lps_3, tracking_df)
print(f"Meilleur joueur: {result['best_player']}, "
      f"Temps de début: {result['best_start_time']:.2f} s, "
      f"Distance moyenne (RMSE): {result['best_distance']:.4f} m")

KeyError: 'player_id'

In [ ]:
def experience2_avec_lsap_sans_interpolation(fusion_df, tracking_df, step=1):

    best_result = {"best_player": None, "best_start_time": None, "best_distance": float("inf")}

    for pid in tracking_df["player_id"].unique():
        joueur_df = tracking_df[tracking_df["player_id"] == pid].sort_values("time")
        if len(joueur_df) < 10:
            continue

        t_min, t_max = joueur_df["time"].min(), joueur_df["time"].max()
        traj_duration = t_max - t_min
        max_start = fusion_df["relative_time"].max() - traj_duration

        for start_time in np.arange(0, max_start, step):
            end_time = start_time + traj_duration

            lps_window = fusion_df[(fusion_df["relative_time"] >= start_time) &
                                   (fusion_df["relative_time"] <= end_time)]

            if len(lps_window) < 10 or len(lps_window) != len(joueur_df):
                continue

            gps_x = joueur_df["x"].values
            gps_y = joueur_df["y"].values
            lps_x = lps_window["x"].values
            lps_y = lps_window["y"].values

            gps_x = (gps_x - gps_x.mean()) / gps_x.std()
            gps_y = (gps_y - gps_y.mean()) / gps_y.std()
            lps_x = (lps_x - lps_x.mean()) / lps_x.std()
            lps_y = (lps_y - lps_y.mean()) / lps_y.std()
            

            cost_matrix = np.sqrt((gps_x - lps_x)**2 + (gps_y - lps_y)**2).mean()
            
            print(cost_matrix.shape)
            row_ind, col_ind = linear_sum_assignment(cost_matrix)
            matched_gps_x = gps_x[row_ind]
            matched_gps_y = gps_y[row_ind]
            matched_lps_x = lps_x[col_ind]
            matched_lps_y = lps_y[col_ind]

            rmse = np.sqrt(np.mean((matched_gps_x - matched_lps_x)**2 + (matched_gps_y - matched_lps_y)**2))

            if rmse < best_result["best_distance"]:
                best_result = {
                    "best_player": pid,
                    "best_start_time": start_time,
                    "best_distance": rmse
                }

    return best_result


In [ ]:
def experience2_avec_lsap_sans_interpolation(fusion_df, tracking_df, step=1.0):
    best_result = {"best_player": None, "best_start_time": None, "best_distance": float("inf")}

    for pid in tracking_df["player_id"].unique():
        joueur_df = tracking_df[tracking_df["player_id"] == pid].sort_values("time")

        if len(joueur_df) < 10:
            continue

        t_min, t_max = joueur_df["time"].min(), joueur_df["time"].max()
        traj_duration = t_max - t_min
        max_start = fusion_df["relative_time"].max() - traj_duration

        for start_time in np.arange(0, max_start, step):
            end_time = start_time + traj_duration

            lps_window = fusion_df[(fusion_df["relative_time"] >= start_time) &
                                   (fusion_df["relative_time"] <= end_time)]

            if len(lps_window) != len(joueur_df):
                continue  # Important pour créer une matrice carrée (même nombre de points)

            # Normalisation
            gps_x = (joueur_df["x"].values - joueur_df["x"].mean()) / joueur_df["x"].std()
            gps_y = (joueur_df["y"].values - joueur_df["y"].mean()) / joueur_df["y"].std()
            lps_x = (lps_window["x"].values - lps_window["x"].mean()) / lps_window["x"].std()
            lps_y = (lps_window["y"].values - lps_window["y"].mean()) / lps_window["y"].std()

            # Création de la matrice de coût : distances euclidiennes pour chaque paire de points
            cost_matrix = np.zeros((len(lps_x), len(gps_x)))

            for i in range(len(lps_x)):
                for j in range(len(gps_x)):
                    dx = gps_x[j] - lps_x[i]
                    dy = gps_y[j] - lps_y[i]
                    cost_matrix[i, j] = np.sqrt(dx**2 + dy**2)

            # Résolution du LSAP
            row_ind, col_ind = linear_sum_assignment(cost_matrix)

            matched_gps_x = gps_x[col_ind]
            matched_gps_y = gps_y[col_ind]
            matched_lps_x = lps_x[row_ind]
            matched_lps_y = lps_y[row_ind]

            # Calcul du RMSE entre les points associés
            rmse = np.sqrt(np.mean((matched_gps_x - matched_lps_x)**2 + (matched_gps_y - matched_lps_y)**2))

            if rmse < best_result["best_distance"]:
                best_result = {
                    "best_player": pid,
                    "best_start_time": start_time,
                    "best_distance": rmse
                }

    return best_result


In [ ]:
cost_matrix.min()

np.float64(0.0576779045043712)

In [ ]:
row_ind

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154])

In [ ]:
col_ind

array([154, 114, 112, 111,  99, 100, 101, 102, 107, 104, 105, 106, 103,
       108, 109, 110, 113, 115, 153, 152, 151, 116, 150, 149, 119, 130,
       126, 125, 122, 123, 132, 136, 133, 139, 140, 142, 143, 144, 141,
       145, 134, 135, 138, 124, 137, 131, 121, 120, 127, 128, 129, 146,
       147, 148, 117, 118,   0,   1,   6,   5,  42,  67,  47,  40,  69,
        48,  50,  49,  38,  39,  53,  59,  82,  81,  80,  79,  76,  77,
        74,  57,  37,  36,  55,  35,  30,  70,  29,  32,  72,  71,  33,
        34,  27,  73,  26,  28,  31,  58,  60,  54,  56,  75,  78,  83,
        84,  85,  86,  87,  88,  90,  89,  91,  92,  93,  94,  95,  96,
        97,  98,  61,  52,  51,  41,  65,  63,   4,   3,   2,   8,   7,
         9,  46,  44,  10,  64,  45,  16,  18,  15,  11,  12,  14,  13,
        17,  19,  21,  20,  43,  22,  23,  68,  62,  24,  25,  66])

In [ ]:
lps_3

,timestamp,latitude_fusion,longitude_fusion,x,y,x_norm,y_norm,datetime,relative_time
0,1702478139700,49.458047,1.062343,559482.518544,6.930462e+06,-1.236547,-1.347295,2023-12-13 14:35:39.700,0.0
1,1702478139800,49.458045,1.062340,559482.359227,6.930462e+06,-1.395864,-1.518088,2023-12-13 14:35:39.800,0.1
2,1702478139900,49.458045,1.062340,559482.335438,6.930462e+06,-1.419652,-1.560713,2023-12-13 14:35:39.900,0.2
3,1702478140000,49.458045,1.062341,559482.379418,6.930462e+06,-1.375673,-1.537862,2023-12-13 14:35:40.000,0.3
4,1702478140100,49.458045,1.062342,559482.455782,6.930462e+06,-1.299309,-1.482542,2023-12-13 14:35:40.100,0.4
...,...,...,...,...,...,...,...,...,...
68442,1702486860300,49.457994,1.062402,559486.697657,6.930456e+06,2.942566,-7.356699,2023-12-13 17:01:00.300,8720.6
68443,1702486860400,49.457993,1.062404,559486.844951,6.930456e+06,3.089860,-7.421624,2023-12-13 17:01:00.400,8720.7
68444,1702486860500,49.457993,1.062406,559487.000895,6.930456e+06,3.245804,-7.473139,2023-12-13 17:01:00.500,8720.8
68445,1702486860600,49.457992,1.062409,559487.172765,6.930456e+06,3.417674,-7.515129,2023-12-13 17:01:00.600,8720.9
